In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import make_scorer, f1_score
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
import joblib

In [ ]:
db1 = pd.read_csv('../data/db1.csv')
db2 = pd.read_csv('../data/db2.csv')

y_train = db1['Diabetes_binary'].replace({2: 1})
X_train = db1.drop('Diabetes_binary', axis=1)

X_test = db2.drop('Diabetes_binary', axis=1)
y_test = db2['Diabetes_binary']


In [35]:
cv = joblib.load("../models/cv_res.pkl")
list(cv)
new_cv = np.delete(cv, [1,2,3,4,5,6,7,8,9])
cv = new_cv
cv

array([0.82229581])

In [ ]:
pipeline = joblib.load("../models/pipeline_side.pkl")
pipeline

In [ ]:
# Check booster-level of XGB
md = XGBClassifier()
print(md.get_params().keys())

In [ ]:
# Too heavy grid
'''par_grid = {
    'model__n_estiators': [200, 400],
    'model__max_depth' : [4,6,8],
    'model__learning_rate': [0.01, 0.02, 0.05, 0.1],
    'model__subsample': [0.7, 1.0],
    'model__colsample_bytree': [0.7, 1.0],
    'model__c': [0.1, 1, 10],
    'model__gamma': ['scale', 'auto']
}'''

# Adjust
par_grid = {
    'model__max_depth': [4,6],
    'model__learning_rate': [0.01, 0.005],
    'model__n_estimators': [200, 400]
}

In [37]:
# Heavy load
'''RDS = RandomizedSearchCV(
    estimator = pipeline,
    param_distributions = par_grid,
    cv = cv,
    verbose = 2,
    n_jobs = 2
)
'''
# Lighter load

RDS = RandomizedSearchCV(
    estimator = pipeline,
    param_distributions = par_grid,
    n_iter = 10,
    scoring = 'f1',
    cv = cv,
    n_jobs = 1,
    verbose = 1
)



In [ ]:
grid = GridSearchCV(
    estimator = pipeline,
    param_grid = par_grid,
    scoring = make_scorer(f1_score),
    cv = cv,
    verbose = 2,
    n_jobs = 1
)

grid.fit(X_train, y_train)

In [ ]:
RDS.fit(X_train, y_train)
# => Computer crash at 8 fits

Fitting 1 folds for each of 8 candidates, totalling 8 fits


TypeError: cannot unpack non-iterable numpy.float64 object

In [ ]:
print("Best params:", grid.best_params_)
print("Best score:", grid.best_score_)

best_model = grid.best_estimator_

joblib.dump(best_model, "models/xgb_stage2_best.pkl")

ADASYN creates synthetic samples in pipeline.
It includes 10 folds => iterated 10 times => many milions samples created.

XGBoost needs more RAM to run

Decrease n_iter won't help.

This is the end for the data by running over 300k rows

Solution (side.py)
    => Remove ADASYN after tuning, oversample after find the best hyperparameter.
    => Retrain final model with ADASYN with best pars